# **Schnorr Digital Signature Algorithm (EC SSA)**

# Setup

btclib is needed: let's install/update it and import straight away some of its functions

In [1]:
!pip install --upgrade btclib

from hashlib import sha256 as hf

from btclib.ecc.number_theory import mod_inv
from btclib.ecc.curve import mult
from btclib.ecc.curve import secp256k1 as ec

Requirement already up-to-date: btclib in /Users/ferdi/Git/myforks/btclib (2020.12)


For this exercise we use secp256k1 as elliptic curve and SHA256 as hash function:

In [2]:
print(ec)

Curve
 p   = FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE FFFFFC2F
 a   = 0
 b   = 7
 x_G = 79BE667E F9DCBBAC 55A06295 CE870B07 029BFCDB 2DCE28D9 59F2815B 16F81798
 y_G = 483ADA77 26A3C465 5DA4FBFC 0E1108A8 FD17B448 A6855419 9C47D08F FB10D4B8
 n   = FFFFFFFF FFFFFFFF FFFFFFFF FFFFFFFE BAAEDCE6 AF48A03B BFD25E8C D0364141
 h = 1


Normally, hf is chosen such that its output size is roughly equal to the size of ec.n, since the overall security of the signature scheme will depend on the smallest of the two; however, the ECDSA standard support all combinations of sizes

In [3]:
print(hf().digest_size)
print(ec.nsize)

32
32


# **Digital Signature Protocol**

## 1. Key generation

Private key (generated elsewhere, a fixed value here):

In [4]:
q = 0x18E14A7B6A307F426A94F8114701E7C8E774E7F9A47E2C2035DB29A206321725
assert 0 < q < ec.n, "Invalid private key"
print("q:", q)
print("Hex(q):", hex(q))

q: 11253563012059685825953619222107823549092147699031672238385790369351542642469
Hex(q): 0x18e14a7b6a307f426a94f8114701e7c8e774e7f9a47e2c2035db29a206321725


Corresponding Public Key:

In [6]:
Q = mult(q)
print("PubKey:", "02" if (Q[1] % 2 == 0) else "03", hex(Q[0]))



PubKey: 02 0x50863ad64a87ae8a2fe83c1af1a8403cb53f53e486d8511dad8a04887e5b2352


##Signature (DSA)

###Message
Message to be signed and its hash value:

In [7]:
msg = "Paolo is afraid of ephemeral random numbers"

h_bytes = hf(msg.encode()).digest()
# hash(msg) must be transformed into an integer modulo ec.n:
h = int.from_bytes(h_bytes, 'big') % ec.n
assert h != 0
print("h:", hex(h))

h: 0x9788fd27b3aafd1bd1591a1158ce2d8bdc37ab4040dddb64e64d17616e69ce2b


### Deterministic Ephemeral key
The Ephemeral key k must be kept secret and never reused
A good choice is to use a deterministic key: 

`k = hf(q||msghd)` 

different for each msg, private because of q

In [8]:
k_bytes = hf(q.to_bytes(32, 'big') + msg.encode()).digest()
k = int.from_bytes(k_bytes, 'big') % ec.n
assert k != 0
print("eph k:", hex(k))

eph k: 0xc2ddd3b2037595043d5892733372cd9c8f18b7b14162cbcd0504ca2f3d5d7b4a


###Signature Algorithm

In [9]:
K = mult(k)

r = K[0] % ec.n
# if r == 0 (extremely unlikely for large ec.n) go back to a different ephemeral key
assert r != 0

s = ((h + r*q)*mod_inv(k, ec.n)) % ec.n
# if s1 == 0 (extremely unlikely for large ec.n) go back to a different ephemeral key
assert s != 0

print("r:", hex(r))
print("s:", hex(s))

r: 0xac32d9a5d37c92d103fff8135ea7032b8e9da4a25f547cda8f2216fe49c1d53f
s: 0xb0f50ced477d3bdc23e7efc01d158402a90f27fe916c5358595c2170136e6b9c


## Signature verification (DSA)

In [10]:
w = mod_inv(s, ec.n)
u = (h*w) %ec.n
v = (r*w) %ec.n
assert u != 0
assert v != 0
U = mult(u)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x %ec.n)


True


## Malleated Signature

In [11]:
sm = ec.n - s
print("     r:", hex(r))
print("   *sm:", hex(sm))

     r: 0xac32d9a5d37c92d103fff8135ea7032b8e9da4a25f547cda8f2216fe49c1d53f
   *sm: 0x4f0af312b882c423dc18103fe2ea7bfc119fb4e81ddc4ce366763d1cbcc7d5a5


Malleated Signature verification:

In [13]:
w = mod_inv(sm, ec.n)
u = (h*w) %ec.n
v = (r*w) %ec.n
assert u != 0
assert v != 0
U = mult(u)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x %ec.n)

True


##Humongous mistake

### Message
Message to be signed and its hash value:

In [14]:
msg2 = "and Paolo is right to be afraid"

h_bytes = hf(msg2.encode()).digest()
# hash(msg) must be transformed into an integer modulo ec.n:
h2 = int.from_bytes(h_bytes, 'big') % ec.n
assert h2 != 0
print("h2:", hex(h2))

h2: 0x7adb91982ec03ef87efcae7f0199aefa231d8855e0bd03319460e58c0bd18049


### The mistake 
Reuse the same ephemeral deterministic key as the previous message:

In [15]:
k2 = k #very bad! Never reuse the same ephemeral key!!!
print("eph k :", hex(k))
print("eph k2:", hex(k2))

eph k : 0xc2ddd3b2037595043d5892733372cd9c8f18b7b14162cbcd0504ca2f3d5d7b4a
eph k2: 0xc2ddd3b2037595043d5892733372cd9c8f18b7b14162cbcd0504ca2f3d5d7b4a


###Signature Algorithm

In [16]:
K2 = mult(k2)

r = K2[0] % ec.n
# if r == 0 (extremely unlikely for large ec.n) go back to a different ephemeral key
assert r != 0

s2 = ((h2 + r*q)*mod_inv(k2, ec.n)) %ec.n
# if s2 == 0 (extremely unlikely for large ec.n) go back to a different ephemeral key
assert s2 != 0

# bitcoin canonical 'low-s' encoding
if s2 > ec.n/2: s2 = ec.n - s2

print(" r:", hex(r))
print("s2:", hex(s2))

 r: 0xac32d9a5d37c92d103fff8135ea7032b8e9da4a25f547cda8f2216fe49c1d53f
s2: 0x4b7371273e661e420991dee123eff736264e5a5854767b7d8e2b70a331b4dca6


### Signature verification

In [17]:
w = mod_inv(s2, ec.n)
u = (h2*w) %ec.n
v = (r*w) %ec.n
assert u != 0
assert v != 0
U = mult(u)
V = mult(v, Q)
x, y = ec.add(U, V)
print(r == x %ec.n)


True


### Exercise
Because of this mistake is now possible to calculate the private key from the 2 signatures. 